In [1]:
import matplotlib.pyplot as plt
import rioxarray as rioxr
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from xclim import sdba
import regionmask
import os

os.chdir('/home/rooda/Dropbox/Proyectos/Puelo PEGH/') 
local  = "/media/rooda/Local Disk/Datasets"

baseline_period = slice("1980-01-01", "2010-12-31")
future_period   = slice("2020-01-01", "2060-12-31")

In [2]:
# Baseline climate
chunks_dict = {"longitude": 5, "latitude": 5}
pp_baseline  = xr.open_dataset("Data/Precipitation/PP_CR2MET_d.nc", chunks = chunks_dict).sel(time = baseline_period) # Precipitation from CR2met v2.0
pp_baseline = pp_baseline.rename({"longitude": "lon", "latitude": "lat"})["pr"]
pp_baseline = pp_baseline.sel(time=~((pp_baseline.time.dt.month == 2) & (pp_baseline.time.dt.day == 29)))
weights  = np.cos(np.deg2rad(pp_baseline.lat))
pp_baseline.attrs['units'] = "mm"

t2m_baseline = xr.open_dataset("Data/Temperature/T2M_CR2MET_d.nc", chunks = chunks_dict).sel(time = baseline_period) # Temperature from CR2met v2.0
t2m_baseline = t2m_baseline.rename({"longitude": "lon", "latitude": "lat", "t2m": "tas"})["tas"]
t2m_baseline = t2m_baseline.sel(time=~((t2m_baseline.time.dt.month == 2) & (t2m_baseline.time.dt.day == 29)))
t2m_baseline.attrs["units"] = "degC"

#factor = xr.open_dataset( rast("Data/Precipitation/PP_factor.tif")
#area = project(area, pp_baseline)
#pp_baseline = crop(pp_baseline, area)
#pp_baseline = pp_baseline*resample(factor,pp_baseline)
#t2m_baseline <- crop(t2m_baseline, area)

In [3]:
# Shapefiles of the main basins 
puelo_basin     = gpd.read_file("GIS/Elevation_bands.shp").to_crs("EPSG:4326")
puelo_basin_m   = regionmask.mask_3D_geopandas(puelo_basin, pp_baseline)

yelcho_basin    = gpd.read_file("GIS/Cuencas/Cuenca_Yelcho.shp").to_crs("EPSG:4326")
yelcho_basin_m  = regionmask.mask_3D_geopandas(yelcho_basin, pp_baseline)

pascua_basin    = gpd.read_file("GIS/Cuencas/Cuenca_Pascua.shp").to_crs("EPSG:4326")
pascua_basin_m  = regionmask.mask_3D_geopandas(pascua_basin, pp_baseline)

aysen_basin     = gpd.read_file("GIS/Cuencas/Cuenca_Aysen.shp").to_crs("EPSG:4326")
aysen_basin_m   = regionmask.mask_3D_geopandas(aysen_basin, pp_baseline)

baker_basin     = gpd.read_file("GIS/Cuencas/Cuenca_Baker.shp").to_crs("EPSG:4326")
baker_basin_m   = regionmask.mask_3D_geopandas(baker_basin, pp_baseline)

# Shapefiles of the secondary basins 
puelo_c_basin   = gpd.read_file("GIS/Cuencas/Cuenca_Pcostera.shp").to_crs("EPSG:4326")
puelo_c_basin_m = regionmask.mask_3D_geopandas(puelo_c_basin, pp_baseline)

aysen_c_basin   = gpd.read_file("GIS/Cuencas/Cuenca_Acostera.shp").to_crs("EPSG:4326")
aysen_c_basin_m = regionmask.mask_3D_geopandas(aysen_c_basin, pp_baseline)

baker_c_basin   = gpd.read_file("GIS/Cuencas/Cuenca_Bcostera.shp").to_crs("EPSG:4326")
baker_c_basin_m = regionmask.mask_3D_geopandas(baker_c_basin, pp_baseline)

In [5]:
directory = os.path.join(local + "/GCMs/Daily/")
gcm_list  = os.listdir(directory)
ssp_list  = ["ssp585"]
chunks_dict_b = {"lon": 10, "lat": 10, "time": 11315}
chunks_dict_f = {"lon": 10, "lat": 10, "time": 14965}
os.chdir(directory) 

In [ ]:
import warnings
warnings.simplefilter("ignore") 

for gcm in gcm_list:
    pp_historical = xr.open_mfdataset(os.path.join(local + "/GCMs/Daily/" + gcm + "/pr_day*_historical*.nc"), concat_dim='time', combine='nested')
    pp_historical = pp_historical.sel(time = baseline_period)["pr"]
    pp_historical = pp_historical.rio.write_crs("epsg:4326", inplace=True)
    pp_historical.coords['lon'] = (pp_historical.coords['lon'] + 180) % 360 - 180
    pp_historical = pp_historical.sortby(pp_historical.lon)
    pp_historical = pp_historical.interp(lat = pp_baseline.lat, lon = pp_baseline.lon)*86400
    pp_historical = pp_historical.sel(time=~((pp_historical.time.dt.month == 2) & (pp_historical.time.dt.day == 29)))
    pp_historical = pp_historical.chunk(chunks_dict_b)
    pp_historical.attrs["units"] = "mm"
    pp_historical["time"] = pp_baseline.time
    
    t2m_historical = xr.open_mfdataset(os.path.join(local + "/GCMs/Daily/" + gcm + "/tas_day*_historical*.nc"), concat_dim='time', combine='nested')
    t2m_historical = t2m_historical.sel(time = baseline_period)["tas"]
    t2m_historical = t2m_historical.rio.write_crs("epsg:4326", inplace=True)
    t2m_historical.coords['lon'] = (t2m_historical.coords['lon'] + 180) % 360 - 180
    t2m_historical = t2m_historical.sortby(t2m_historical.lon)
    t2m_historical = t2m_historical.interp(lat = t2m_baseline.lat, lon = t2m_baseline.lon)-273.15
    t2m_historical = t2m_historical.sel(time=~((t2m_historical.time.dt.month == 2) & (t2m_historical.time.dt.day == 29)))
    t2m_historical = t2m_historical.chunk(chunks_dict_b)
    t2m_historical.attrs["units"] = "degC"
    t2m_historical["time"] = t2m_baseline.time
    print(gcm)
    
    for ssp in ssp_list:
        pp_model_ssp = xr.open_mfdataset(os.path.join(local + "/GCMs/Daily/" + gcm + "/pr_day*" + ssp + "*.nc"), chunks = None, concat_dim='time', combine='nested')
        pp_model_ssp = pp_model_ssp.sel(time = future_period)["pr"]
        pp_model_ssp = pp_model_ssp.rio.write_crs("epsg:4326", inplace=True)
        pp_model_ssp.coords['lon'] = (pp_model_ssp.coords['lon'] + 180) % 360 - 180
        pp_model_ssp = pp_model_ssp.sortby(pp_model_ssp.lon)
        pp_model_ssp = pp_model_ssp.interp(lat = pp_baseline.lat, lon = pp_baseline.lon)*86400
        pp_model_ssp = pp_model_ssp.sel(time=~((pp_model_ssp.time.dt.month == 2) & (pp_model_ssp.time.dt.day == 29)))
        pp_model_ssp = pp_model_ssp.chunk(chunks_dict_f)
        pp_model_ssp.attrs["units"] = "mm"
        
        t2m_model_ssp = xr.open_mfdataset(os.path.join(local + "/GCMs/Daily/" + gcm + "/tas_day*" + ssp + "*.nc"), chunks = chunks_dict, concat_dim='time', combine='nested')
        t2m_model_ssp = t2m_model_ssp.sel(time = future_period)["tas"]
        t2m_model_ssp = t2m_model_ssp.rio.write_crs("epsg:4326", inplace=True)
        t2m_model_ssp.coords['lon'] = (t2m_model_ssp.coords['lon'] + 180) % 360 - 180
        t2m_model_ssp = t2m_model_ssp.sortby(t2m_model_ssp.lon)
        t2m_model_ssp = t2m_model_ssp.interp(lat = t2m_baseline.lat, lon = t2m_baseline.lon)-273.15
        t2m_model_ssp = t2m_model_ssp.sel(time=~((t2m_model_ssp.time.dt.month == 2) & (t2m_model_ssp.time.dt.day == 29)))
        t2m_model_ssp = t2m_model_ssp.chunk(chunks_dict_f)
        t2m_model_ssp.attrs["units"] = "degC"
        
        qdm_pp  = sdba.adjustment.QuantileDeltaMapping.train(pp_baseline, pp_historical, kind = "*", nquantiles=15, group="time")
        qdm_t2m = sdba.adjustment.QuantileDeltaMapping.train(ref = t2m_baseline, hist = t2m_historical, kind = "+", nquantiles=15, group="time")
        pp_model_ssp  = qdm_pp.adjust(pp_model_ssp, extrapolation="constant", interp="nearest").rename("pr").drop_vars('spatial_ref')
        t2m_model_ssp = qdm_t2m.adjust(t2m_model_ssp, extrapolation="constant", interp="nearest").rename("tas").drop_vars(['spatial_ref', "height"], errors='ignore')
        
        local_weap = "/home/rooda/Dropbox/Proyectos/Puelo PEGH/WEAP/Projections/"
        pp_puelo_ssp_df     = pp_model_ssp.weighted(puelo_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_puelo_ssp_df    = t2m_model_ssp.weighted(puelo_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        pp_puelo_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_puelo_" + gcm + "_" + ssp + ".csv"))
        tas_puelo_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_puelo_" + gcm + "_" + ssp + ".csv"))
        print("puelo")
        pp_yelcho_ssp_df    = pp_model_ssp.weighted(yelcho_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_yelcho_ssp_df   = t2m_model_ssp.weighted(yelcho_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        pp_yelcho_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_yelcho_" + gcm + "_" + ssp + ".csv"))
        tas_yelcho_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_yelcho_" + gcm + "_" + ssp + ".csv"))
        print("yelcho")
        pp_pascua_ssp_df    = pp_model_ssp.weighted(pascua_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_pascua_ssp_df   = t2m_model_ssp.weighted(pascua_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        pp_pascua_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_pascua_" + gcm + "_" + ssp + ".csv"))
        tas_pascua_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_pascua_" + gcm + "_" + ssp + ".csv"))
        print("pascua")
        pp_aysen_ssp_df     = pp_model_ssp.weighted(aysen_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_aysen_ssp_df    = t2m_model_ssp.weighted(aysen_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        pp_aysen_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_aysen_" + gcm + "_" + ssp + ".csv"))
        tas_aysen_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_aysen_" + gcm + "_" + ssp + ".csv"))
        print("aysen")
        pp_baker_ssp_df     = pp_model_ssp.weighted(baker_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_baker_ssp_df    = t2m_model_ssp.weighted(baker_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        pp_baker_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_baker_" + gcm + "_" + ssp + ".csv"))
        tas_baker_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_baker_" + gcm + "_" + ssp + ".csv"))
        print("baker")
        pp_puelo_c_ssp_df   = pp_model_ssp.weighted(puelo_c_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_puelo_c_ssp_df  = t2m_model_ssp.weighted(puelo_c_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        pp_puelo_c_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_puelo_c_" + gcm + "_" + ssp + ".csv"))
        tas_puelo_c_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_puelo_c_" + gcm + "_" + ssp + ".csv"))
        print("puelo_c")
        pp_aysen_c_ssp_df   = pp_model_ssp.weighted(aysen_c_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_aysen_c_ssp_df  = t2m_model_ssp.weighted(aysen_c_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()  
        pp_aysen_c_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_aysen_c_" + gcm + "_" + ssp + ".csv"))
        tas_aysen_c_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_aysen_c_" + gcm + "_" + ssp + ".csv"))
        print("aysen_c")
        pp_baker_c_ssp_df   = pp_model_ssp.weighted(baker_c_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        tas_baker_c_ssp_df  = t2m_model_ssp.weighted(baker_c_basin_m*weights).mean(dim=("lat", "lon")).to_dataframe().unstack()
        pp_baker_c_ssp_df.round(1).to_csv(os.path.join(local_weap + "pr_baker_c_" + gcm + "_" + ssp + ".csv"))
        tas_baker_c_ssp_df.round(2).to_csv(os.path.join(local_weap + "tas_baker_c_" + gcm + "_" + ssp + ".csv"))
        print("baker_c")
        #pp_model_ssp.to_netcdf(os.path.join(local + "/GCMs/Daily/" + gcm + "/subset_pr_day_" + ssp + ".nc"))
        #t2m_model_ssp.to_netcdf(os.path.join(local + "/GCMs/Daily/" + gcm + "/subset_tas_day_" + ssp + ".nc"))
        
        print(ssp)

CanESM5


In [37]:
        t2m_model_ssp = qdm_t2m.adjust(t2m_model_ssp, extrapolation="constant", interp="nearest").rename("tas").drop_vars(['spatial_ref', "height"], errors='ignore')
t2m_model_ssp

,Array,Chunk
Bytes,615.85 MiB,1.43 MiB
Shape,"(174, 62, 14965)","(5, 5, 14965)"
Count,17927 Tasks,455 Chunks
Type,float32,numpy.ndarray
